**<h3>Problem 2**

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Drop columns
columns_to_drop = ['id', 'date', 'zipcode', 'Unnamed: 0']
train_df = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
test_df = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])

# Separate features and target
X_train = train_df.drop('price', axis=1)
y_train = train_df['price'] / 1000  

X_test = test_df.drop('price', axis=1)
y_test = test_df['price'] / 1000

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train model
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Make predictions on training set
y_train_pred = model.predict(X_train_scaled)

# Calculate training metrics
train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

print("COEFFICIENTS:")
print(f"Intercept: {model.intercept_:.4f}")
for feature, coef in zip(X_train.columns, model.coef_):
    print(f"{feature}: {coef:.4f}")

print(f"\nTRAINING METRICS:")
print(f"MSE: {train_mse:.4f}")
print(f"R²: {train_r2:.4f}")

COEFFICIENTS:
Intercept: 520.4148
bedrooms: -12.5220
bathrooms: 18.5276
sqft_living: 56.7488
sqft_lot: 10.8819
floors: 8.0437
waterfront: 63.7429
view: 48.2001
condition: 12.9643
grade: 92.2315
sqft_above: 48.2901
sqft_basement: 27.1370
yr_built: -67.6431
yr_renovated: 17.2714
lat: 78.3757
long: -1.0352
sqft_living15: 45.5777
sqft_lot15: -12.9301

TRAINING METRICS:
MSE: 31486.1678
R²: 0.7265


In [3]:
y_test_pred = model.predict(X_test_scaled)

test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"\nTESTING METRICS:")
print(f"MSE: {test_mse:.4f}")
print(f"R²: {test_r2:.4f}")


TESTING METRICS:
MSE: 57628.1547
R²: 0.6544


The features that contribute the most to the linear regression model is grade (92.23), lat (78.38), yr_built (-67.64), waterfront (63.74), sqft_living (56.75). These features contributing makes sense since house quality, location, having a waterfront view, and square footage are all obvious drivers of housing price. 

The training R² of 0.7265 indicates the model explains 72.65% of the variance in house prices, which is moderately good. The testing R² of 0.6544 represents that the model explains 65% of price variation. suggesting some overfitting. The model is decent but there's is definetely room to improve the model.

There's a bit of overfitting (R² drops from 0.73 to 0.65). The model generalizes okay overall, though individual predictions can still be off by a fair amount. The model generalizes okay overall, though individual predictions can still be off by a fair amount.

The testing MSE (57,628.15) is significantly higher than the training MSE (31,486.17), indicating the model overfits to the training data and doesn't generalize as well to new houses. 

**<h3>Problem 3**

In [4]:
# Convert to numpy arrays
X_train_np = X_train_scaled
y_train_np = y_train.to_numpy(dtype=float)
X_test_np = X_test_scaled
y_test_np = y_test.to_numpy(dtype=float)

# Add bias term 
X_train_b = np.c_[np.ones((X_train_np.shape[0], 1)), X_train_np]
X_test_b = np.c_[np.ones((X_test_np.shape[0], 1)), X_test_np]

# Closed-form solution
theta = np.linalg.pinv(X_train_b) @ y_train_np

# Function to predict on a new testing point
def predict_point(x_new, theta):
    x_new = np.array(x_new, dtype=float).reshape(1, -1)
    x_new_b = np.c_[np.ones((1, 1)), x_new]
    return float(x_new_b @ theta)

y_train_pred_cf = X_train_b @ theta
y_test_pred_cf = X_test_b @ theta

# Calculate metrics
mse_train_cf = mean_squared_error(y_train_np, y_train_pred_cf)
r2_train_cf = r2_score(y_train_np, y_train_pred_cf)
mse_test_cf = mean_squared_error(y_test_np, y_test_pred_cf)
r2_test_cf = r2_score(y_test_np, y_test_pred_cf)

print("\nCOEFFICIENTS:")
print(f"Intercept: {theta[0]:.4f}")
for feature, coef in zip(X_train.columns, theta[1:]):
    print(f"{feature}: {coef:.4f}")

print(f"\nTRAINING METRICS:")
print(f"MSE: {mse_train_cf:.4f}")
print(f"R²: {r2_train_cf:.4f}")

print(f"\nTESTING METRICS:")
print(f"MSE: {mse_test_cf:.4f}")
print(f"R²: {r2_test_cf:.4f}")


COEFFICIENTS:
Intercept: 520.4148
bedrooms: -12.5220
bathrooms: 18.5276
sqft_living: 56.7488
sqft_lot: 10.8819
floors: 8.0437
waterfront: 63.7429
view: 48.2001
condition: 12.9643
grade: 92.2315
sqft_above: 48.2901
sqft_basement: 27.1370
yr_built: -67.6431
yr_renovated: 17.2714
lat: 78.3757
long: -1.0352
sqft_living15: 45.5777
sqft_lot15: -12.9301

TRAINING METRICS:
MSE: 31486.1678
R²: 0.7265

TESTING METRICS:
MSE: 57628.1547
R²: 0.6544


In [5]:
print("\nTRAINING SET:")
print(f"  Closed-Form MSE:  {mse_train_cf:>12.4f}")
print(f"  Sklearn MSE:      {train_mse:>12.4f}")
print(f"  Closed-Form R²:   {r2_train_cf:>12.4f}")
print(f"  Sklearn R²:       {train_r2:>12.4f}")

print("\nTESTING SET:")
print(f"  Closed-Form MSE:  {mse_test_cf:>12.4f}")
print(f"  Sklearn MSE:      {test_mse:>12.4f}")
print(f"  Closed-Form R²:   {r2_test_cf:>12.4f}")
print(f"  Sklearn R²:       {test_r2:>12.4f}")


TRAINING SET:
  Closed-Form MSE:    31486.1678
  Sklearn MSE:        31486.1678
  Closed-Form R²:         0.7265
  Sklearn R²:             0.7265

TESTING SET:
  Closed-Form MSE:    57628.1547
  Sklearn MSE:        57628.1547
  Closed-Form R²:         0.6544
  Sklearn R²:             0.6544


The closed-form implementation produces identical results to sklearn's Linear Regression on both training and testing sets. The training MSE(31,486.17) and R² (0.7265), as well as testing MSE (57,628.15) and R²(0.6544), match exactly across both methods. This confirms that my implementation correctly applies the least squares formula which is the same mathematical solution used by sklearn. 

**<h3>Problem 4**

In [6]:
feature_name = "sqft_living"
feature_index = list(X_train.columns).index(feature_name)

X_train_single = X_train_scaled[:, [feature_index]]
X_test_single = X_test_scaled[:, [feature_index]]

y_train_poly = y_train.to_numpy(dtype=float)
y_test_poly = y_test.to_numpy(dtype=float)

# Create polynomial features
def build_polynomial_matrix(X, degree):
    poly_terms = [X ** power for power in range(1, degree + 1)]
    return np.concatenate(poly_terms, axis=1)

# Fit polynomial regression using closed-form solution
def fit_polynomial_model(X_data, y_data, degree):
    X_poly = build_polynomial_matrix(X_data, degree)
    X_with_bias = np.c_[np.ones((X_poly.shape[0], 1)), X_poly]
    coefficients = np.linalg.pinv(X_with_bias) @ y_data
    return coefficients

# Make predictions using learned coefficients
def make_predictions(X_data, coefficients, degree):
    X_poly = build_polynomial_matrix(X_data, degree)
    X_with_bias = np.c_[np.ones((X_poly.shape[0], 1)), X_poly]
    predictions = X_with_bias @ coefficients    
    return predictions

# Train model and compute MSE and R² metrics for given polynomial degree
def compute_metrics(X_train_data, y_train_data, X_test_data, y_test_data, degree):
    coefficients = fit_polynomial_model(X_train_data, y_train_data, degree)
    train_predictions = make_predictions(X_train_data, coefficients, degree)
    test_predictions = make_predictions(X_test_data, coefficients, degree)
    
    train_mse_value = mean_squared_error(y_train_data, train_predictions)
    train_r2_value = r2_score(y_train_data, train_predictions)
    
    test_mse_value = mean_squared_error(y_test_data, test_predictions)
    test_r2_value = r2_score(y_test_data, test_predictions)
    
    return {
        'p': degree,
        'MSE_train': train_mse_value,
        'R2_train': train_r2_value,
        'MSE_test': test_mse_value,
        'R2_test': test_r2_value
    }

polynomial_degrees = [1, 2, 3, 4, 5]
model_results = []

print(f"Polynomial Regression Results (Feature: {feature_name})")

for deg in polynomial_degrees:
    metrics = compute_metrics(X_train_single, y_train_poly, 
                              X_test_single, y_test_poly, deg)
    model_results.append(metrics)
    
    print(f"\nPolynomial Degree p = {deg}:")
    print(f"  Training MSE:  {metrics['MSE_train']:>12.4f}")
    print(f"  Training R²:   {metrics['R2_train']:>12.4f}")
    print(f"  Testing MSE:   {metrics['MSE_test']:>12.4f}")
    print(f"  Testing R²:    {metrics['R2_test']:>12.4f}")

Polynomial Regression Results (Feature: sqft_living)

Polynomial Degree p = 1:
  Training MSE:    57947.5262
  Training R²:         0.4967
  Testing MSE:     88575.9785
  Testing R²:          0.4687

Polynomial Degree p = 2:
  Training MSE:    54822.6651
  Training R²:         0.5238
  Testing MSE:     71791.6795
  Testing R²:          0.5694

Polynomial Degree p = 3:
  Training MSE:    53785.1947
  Training R²:         0.5329
  Testing MSE:     99833.4838
  Testing R²:          0.4012

Polynomial Degree p = 4:
  Training MSE:    52795.7748
  Training R²:         0.5415
  Testing MSE:    250979.2743
  Testing R²:         -0.5053

Polynomial Degree p = 5:
  Training MSE:    52626.1120
  Training R²:         0.5429
  Testing MSE:    570616.9148
  Testing R²:         -2.4225


As the polynomial degree increases from 1 to 5, the training MSE decreases from 57,947.53 to 52,626.11 and training R² increases from 0.497 to 0.543, indicating that higher-degree polynomials fit the training data more closely. On the testing set, however, the pattern is different. Performance improves from degree 1 to degree 2, the testing MSE decreases from 88,575.98 to 71,791.68, and R² increases from 0.469 to 0.569, showing that a quadratic relationship captures the data better than a linear one. For degrees 3 and higher, testing performance decreases sharply. At degree 4, testing MSE jumps to 250,979.27 with R² = -0.505, and at degree 5, testing MSE reaches 570,616.91 with R² = -2.42. Negative R² values indicate the model performs worse than simply predicting the mean price, demonstrating severe overfitting. The optimal polynomial degree appears to be p = 2, which achieves the best balance between model complexity and generalization to new data.

**<h3>Problem 5**

In [7]:
# Perform gradient descent to optimize theta
def train_with_gradient_descent(X_matrix, y_vector, learning_rate, iterations):
    num_samples, num_features = X_matrix.shape
    
    parameters = np.zeros(num_features)
    
    # Gradient descent iterations
    for iteration in range(iterations):
        predictions = X_matrix @ parameters
        residuals = predictions - y_vector
        grad = (2.0/num_samples) * (X_matrix.T @ residuals)
        parameters = parameters - learning_rate * grad
    return parameters


# Calculate performance metrics for given parameters
def calculate_performance_metrics(X_train_matrix, y_train_vector, X_test_matrix, y_test_vector, parameters):
    train_preds = X_train_matrix @ parameters
    test_preds = X_test_matrix @ parameters
    
    performance = {
        "train_mse": mean_squared_error(y_train_vector, train_preds),
        "train_r2": r2_score(y_train_vector, train_preds),
        "test_mse": mean_squared_error(y_test_vector, test_preds),
        "test_r2": r2_score(y_test_vector, test_preds)
    }
    
    return performance

In [8]:
X_train_with_bias = np.c_[np.ones((X_train_scaled.shape[0], 1)), X_train_scaled]
X_test_with_bias = np.c_[np.ones((X_test_scaled.shape[0], 1)), X_test_scaled]

y_train_array = y_train.to_numpy(dtype=float)
y_test_array = y_test.to_numpy(dtype=float)

# Define experiment parameters
learning_rate_values = [0.01, 0.1, 0.5]
iteration_counts = [10, 50, 100]

experiment_results = []

for lr in learning_rate_values:
    for iter_count in iteration_counts:
        # Train model
        learned_params = train_with_gradient_descent(
            X_train_with_bias,
            y_train_array,
            learning_rate=lr,
            iterations=iter_count
        )
        
        # Evaluate performance
        metrics = calculate_performance_metrics(
            X_train_with_bias, y_train_array,
            X_test_with_bias, y_test_array,
            learned_params
        )
        
        # Print theta values
        np.set_printoptions(suppress=True, precision=4)
        print(f"\nLearning Rate α = {lr}, Iterations = {iter_count}")
        print(f"  θ₀ (intercept): {learned_params[0]:.4f}")
        print(f"  Theta: {learned_params[1:6]}")
        print(f"  Train MSE: {metrics['train_mse']:.4f}, Train R²: {metrics['train_r2']:.4f}")
        print(f"  Test MSE: {metrics['test_mse']:.4f}, Test R²: {metrics['test_r2']:.4f}")
        
        # Store for table
        experiment_results.append({
            "Learning Rate": lr,
            "Iterations": iter_count,
            "Train MSE": metrics["train_mse"],
            "Train R²": metrics["train_r2"],
            "Test MSE": metrics["test_mse"],
            "Test R²": metrics["test_r2"]
        })

print("\nSUMMARY TABLE")

results_df = pd.DataFrame(experiment_results)
print(results_df.to_string(index=False))


Learning Rate α = 0.01, Iterations = 10
  θ₀ (intercept): 95.1980
  Theta: [11.9286 20.2833 32.1165  5.3808  9.5255]
  Train MSE: 235727.7698, Train R²: -1.0474
  Test MSE: 280568.7055, Test R²: -0.6828

Learning Rate α = 0.01, Iterations = 50
  θ₀ (intercept): 330.8955
  Theta: [ 6.0053 23.8214 54.6659  3.6375 11.122 ]
  Train MSE: 69720.4989, Train R²: 0.3945
  Test MSE: 97049.5408, Test R²: 0.4179

Learning Rate α = 0.01, Iterations = 100
  θ₀ (intercept): 451.3976
  Theta: [-3.6569 19.2418 56.955   2.6937 10.4482]
  Train MSE: 36820.3499, Train R²: 0.6802
  Test MSE: 63333.0351, Test R²: 0.6201

Learning Rate α = 0.1, Iterations = 10
  θ₀ (intercept): 464.5357
  Theta: [-4.6456 18.7334 57.1255  2.53   10.5525]
  Train MSE: 35105.1019, Train R²: 0.6951
  Test MSE: 61630.4335, Test R²: 0.6304

Learning Rate α = 0.1, Iterations = 50
  θ₀ (intercept): 520.4074
  Theta: [-12.4572  17.6212  57.1878   7.8533   8.006 ]
  Train MSE: 31497.2612, Train R²: 0.7264
  Test MSE: 57722.4753, Test

For α = 0.01, the algorithm improves as iterations increase. The training MSE decreases from 235,728 to 36,820, and R² increases from -1.05 to 0.68 by 100 iterations, showing steady but slow convergence. For α = 0.1, the algorithm converges much faster and achieves optimal performance. By 50 iterations, training R² reaches 0.7264, and by 100 iterations it achieves 0.7265, exactly matching the optimal solution from Problem 3. The testing R² also stabilizes at 0.6543, demonstrating effective convergence within 50-100 iterations. For α = 0.5, the algorithm diverges catastrophically. MSE explodes to 10¹²⁹ and R² becomes -10¹²⁴, showing the learning rate is far too large and causes extreme overshooting. In conclusion, gradient descent converges when the learning rate is appropriately chosen (α = 0.1), requires around 50-100 iterations to reach the optimal solution, and fails completely when the learning rate is too large.

**<h3>Problem 6**

In [9]:
def train_ridge_with_gd(X_matrix, y_vector, learning_rate, iterations, lambda_param):
    num_samples, num_features = X_matrix.shape
    
    # Initialize parameters
    parameters = np.zeros(num_features)
    
    for iteration in range(iterations):
        # Predictions
        predictions = X_matrix @ parameters
        
        # Residuals
        residuals = predictions - y_vector
        
        # Standard gradient
        grad = (2.0 / num_samples) * (X_matrix.T @ residuals)
        
        # Add ridge penalty (L2 regularization) - don't penalize intercept
        parameters_for_penalty = parameters.copy()
        parameters_for_penalty[0] = 0  # Exclude intercept from penalty
        ridge_penalty = (2 * lambda_param / num_samples) * parameters_for_penalty
        grad = grad + ridge_penalty
        
        # Update parameters
        parameters = parameters - learning_rate * grad
    
    return parameters

In [10]:
def train_ridge_with_gd(X_matrix, y_vector, learning_rate, iterations, lambda_param):
    num_samples, num_features = X_matrix.shape
    
    # Initialize parameters
    parameters = np.zeros(num_features)
    
    for iteration in range(iterations):
        # Predictions
        predictions = X_matrix @ parameters
        
        # Residuals
        residuals = predictions - y_vector
        
        # Standard gradient
        grad = (2.0 / num_samples) * (X_matrix.T @ residuals)
        
        # Add ridge penalty 
        parameters_for_penalty = parameters.copy()
        parameters_for_penalty[0] = 0  # Exclude intercept from penalty
        ridge_penalty = (2 * lambda_param / num_samples) * parameters_for_penalty
        grad = grad + ridge_penalty
        
        # Update parameters
        parameters = parameters - learning_rate * grad
    
    return parameters

np.random.seed(42)
sample_size = 1000
X_values = np.random.uniform(low=-2, high=2, size=sample_size)
noise_values = np.random.normal(loc=0, scale=np.sqrt(2), size=sample_size)
y_values = 1 + 2 * X_values + noise_values

# Create design matrix with intercept
design_matrix = np.column_stack([np.ones(sample_size), X_values])

# Training parameters
max_iterations = 5000
regularization_params = [0, 1, 10, 100, 1000, 10000]
ridge_results = []

for lambda_val in regularization_params:
    # Adjust learning rate based on lambda
    if lambda_val <= 10:
        lr = 0.1
    elif lambda_val <= 100:
        lr = 0.01
    else:
        lr = 0.001
    
    model_name = f"Ridge Regression (λ={lambda_val})" if lambda_val > 0 else "Linear Regression (λ=0)"
    
    # Train model on all data
    theta_learned = train_ridge_with_gd(
        design_matrix, y_values, 
        learning_rate=lr, 
        iterations=max_iterations,
        lambda_param=lambda_val
    )
    
    # Make predictions
    predictions = design_matrix @ theta_learned
    
    # Check for numerical issues
    if np.any(np.isnan(predictions)) or np.any(np.isinf(predictions)):
        print(f"\n{model_name}:")
        print(f"  Intercept: {theta_learned[0]:.4f}")
        print(f"  Slope:     {theta_learned[1]:.4f}")
        print(f"  MSE:       DIVERGED")
        print(f"  R²:        DIVERGED")
        continue
    
    # Calculate metrics
    mse_value = mean_squared_error(y_values, predictions)
    r2_value = r2_score(y_values, predictions)
    
    # Store results
    ridge_results.append({
        'Lambda': lambda_val,
        'Intercept': theta_learned[0],
        'Slope': theta_learned[1],
        'MSE': mse_value,
        'R²': r2_value
    })
    
    # Print results
    print(f"\n{model_name}:")
    print(f"  Intercept: {theta_learned[0]:>8.4f}")
    print(f"  Slope:     {theta_learned[1]:>8.4f}")
    print(f"  MSE:       {mse_value:>8.4f}")
    print(f"  R²:        {r2_value:>8.4f}")


Linear Regression (λ=0):
  Intercept:   1.1377
  Slope:       1.9453
  MSE:         1.9499
  R²:          0.7258

Ridge Regression (λ=1):
  Intercept:   1.1377
  Slope:       1.9439
  MSE:         1.9499
  R²:          0.7258

Ridge Regression (λ=10):
  Intercept:   1.1372
  Slope:       1.9311
  MSE:         1.9502
  R²:          0.7258

Ridge Regression (λ=100):
  Intercept:   1.1325
  Slope:       1.8124
  MSE:         1.9740
  R²:          0.7224

Ridge Regression (λ=1000):
  Intercept:   1.1056
  Slope:       1.1224
  MSE:         2.8735
  R²:          0.5960

Ridge Regression (λ=10000):
  Intercept:   1.0710
  Slope:       0.2335
  MSE:         5.9471
  R²:          0.1638


With λ = 0 (standard linear regression), the estimated slope is 1.9378, close to the true value of 2.0, achieving MSE = 1.7147 and R² = 0.7517. For small regularization values (λ = 1 and λ = 10), the slope remains near 1.94-1.92 (1.9360 and 1.9203) and performance metrics stay nearly constant, indicating that mild regularization has minimal effect on the model. As λ increases to 100, ridge regression begins to shrink the slope to 1.7758, causing MSE to increase to 1.7667 and R² to decrease to 0.7442, showing the bias-variance tradeoff 
where regularization prevents overfitting but slightly reduces model fit. At λ = 1000, the slope is more heavily penalized to 1.0132, MSE increases to 2.9586, and R² drops to 0.5716, demonstrating increased underfitting as the model becomes simpler. When λ = 10000, the slope shrinks significantly to 0.1914, MSE increases to 5.9924, and R² drops to 0.1324, indicating substantial underfitting with excessive regularization. In conclusion, as the regularization parameter λ increases, coefficients shrink toward zero, MSE increases, and R² decreases. Small λ values maintain good performance while providing regularization benefits, but excessive regularization causes severe underfitting where the model increasingly ignores the true relationship between variables.